# Neural Network Weight Optimisation Using Particle Swarm Optimisation

## Tensorflow and PyGAD

### Importing Libararies

In [29]:
import sklearn
import pandas as pd
import pygad
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from pygad.kerasga import KerasGA
from sklearn.metrics.pairwise import cosine_similarity
import random

### Loading and Preprocessing the Data

In [31]:
df = pd.read_csv('processed_data.csv')

In [32]:
df = df.drop('Unnamed: 0',axis = 1)

In [33]:
df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,target
0,25,1,49,4,1.6,1,0,1,0,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,1,0


In [34]:
df = pd.get_dummies(df,columns = ['Education'],drop_first = True)

In [35]:
X = df.drop('target',axis = 1)
y = df['target']

### Splitting it into train and test and converting target into categorical data

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, 
                                                    test_size=0.33, 
                                                    random_state=42,
                                                    shuffle = True)

In [38]:
y_train=np_utils.to_categorical(y_train,num_classes=2)
y_test=np_utils.to_categorical(y_test,num_classes=2)

### Defining a basic tensorflow ANN 

In [39]:
model = Sequential([tf.keras.layers.Dense(12,input_shape = (12,),activation = 'relu'),
                    tf.keras.layers.Dense(8,activation = 'relu'),
                    tf.keras.layers.Dense(16, activation = 'relu'),
                    tf.keras.layers.Dense(8,activation = 'relu'),
                    tf.keras.layers.Dense(2,activation = 'softmax')
                   ])

In [40]:
model.compile(optimizer = 'adam', metrics = ['accuracy'],loss = 'categorical_crossentropy')

### Fitness function 

In [42]:
def fitness_func(solution, sol_idx):
    global X_train, y_train, keras_ga, model2
    
    model_weights_matrix = pygad.kerasga.model_weights_as_matrix(model = model,weights_vector = solution)
    
    model.set_weights(weights = model_weights_matrix)
    
    predictions = model.predict(X_train)
    
    loss = tf.keras.losses.CategoricalCrossentropy()
    
    solution_fitness = 1.0/(loss(y_train,predictions).numpy() + 0.0000000001)
    
    return solution_fitness

In [43]:
def set_final_weights(weights):
    model_weights_matrix = pygad.kerasga.model_weights_as_matrix(model = model,weights_vector = weights)
    
    model.set_weights(weights = model_weights_matrix)

In [50]:
def pso():
    for i in range (number_of_particles):
        particle_fitness = fitness_func(positions[i],i)
        if (particle_fitness > p_best['fitness'][0][i]):
            p_best['fitness'][0][i] = particle_fitness
            p_best['weights'][i] = positions[i]
        if (particle_fitness > g_best['fitness']):
            g_best['fitness'] = particle_fitness
            g_best['weights'] = positions[i]
            
    for i in range (number_of_particles):
    
        r1 = np.random.random()
        r2 = np.random.random()

        velocities[i] = W*velocities[0][i] + c1*r1*(p_best['weights'][i]-positions[i]) + c2*r2*(g_best['weights'] - positions[i])

        positions[i] = velocities[i] + positions[i]

### Initialising the parameters for particle swarm optimisation

In [44]:
# Get the shapes of the model's weights
weights_shapes = [w.shape for w in model.get_weights()]

# Compute the total number of weights
num_weights = np.sum([np.prod(s) for s in weights_shapes])

In [45]:
number_of_particles = 10

In [46]:
positions = np.random.uniform(size = (number_of_particles,num_weights))

In [47]:
velocities = np.random.uniform(size = (number_of_particles,num_weights))

In [48]:
p_best = {'fitness' : np.zeros((1,number_of_particles)),'weights' : np.zeros((number_of_particles,num_weights))}
g_best = {'fitness': 0, 'weights' : np.zeros((number_of_particles,num_weights))}

In [49]:
W = 0.8
c1 = 2
c2 = 2

### Running the algorithm for 10 iterations

In [51]:
for i in range(10):
    pso()

92/92 [==============================] - 0s 366us/step


### Setting the global best weights to the model

In [52]:
set_final_weights(g_best['weights'])
g_best

{'fitness': 4.234377885887393,
 'weights': array([ -9.30982296, -10.73382632,  -7.94956472,  -8.38460092,
        -11.72722145, -11.17515399,  -8.84173859,  -8.42466603,
        -10.16411751, -10.25180941, -10.35000101,  -8.26346794,
         -9.09843099,  -9.22479393,  -8.82355542, -10.7006621 ,
         -8.74528055, -11.36977521,  -9.69510557,  -9.40341232,
         -9.78472478,  -8.03831941,  -7.68995411, -10.26667403,
         -9.08205289,  -8.91551725,  -8.48526448,  -9.97823185,
         -9.53190702, -11.46795075,  -9.16133128,  -9.8128345 ,
         -8.98014781,  -8.67992006, -11.01272663,  -9.43530408,
         -9.49121506,  -8.34415843, -10.91580473,  -9.92008134,
        -10.29351358,  -9.35574598,  -9.74167863,  -8.38498269,
         -7.50575959,  -7.82369718, -10.72137015,  -9.73604237,
         -9.86836021, -11.33764607, -10.08355885, -11.3422245 ,
        -10.64152173,  -8.14118141, -10.6115135 ,  -9.62847355,
         -9.05937513,  -9.26328014,  -7.83869327, -11.55700159

### Evaluating the model against the test data

In [53]:
model.evaluate(X_test,y_test)

45/45 [==============================] - 0s 540us/step - loss: 0.2171 - accuracy: 0.9437


[0.2170722931623459, 0.9437109231948853]

## Results

Number of particles: 10

Number of iterations : 10

Inertial weight: 0.8

c1 : 2

c2 : 2




Fitness of global best : 4.234

Testing accuracy : 94%